QR Code Based Smart Door Lock 


In [ ]:
import cv2 
import numpy  as np 
from pyzbar.pyzbar import decode 
from cvzone import cornerRect, putTextRect
import pandas as pd  
import re 
import time
from datetime import datetime 
from cvzone.SerialModule import SerialObject
from cvzone.FPS import FPS 

arduino = SerialObject("COM7", digits=1)

fpsReader = FPS(avgCount=30)

cap = cv2.VideoCapture(0)

flag = True
counter = 0

# Read the Excel file
excel_file_path = r'C:\Users\hoque\Desktop\OpenCV\VSCode\QR  Scanner Database\QR_Database.xlsx'  # Update with the path to your Excel file
df = pd.read_excel(excel_file_path)

#Log File Info 
log_file_path = r'C:\Users\hoque\Desktop\OpenCV\VSCode\QR  Scanner Database\QR_Log.xlsx'
log_df = pd.read_excel(log_file_path)

# Convert the 'ID' column to a list
id_list = df['ID'].tolist()
#print(id_list)

#Configure the background image 
imgBackground = cv2.imread(r"C:\Users\hoque\Desktop\OpenCV\VSCode\Files\background2.jpg")
imgBackground = cv2.resize(imgBackground, (1000,500))

#Function for logging information 
def log_event(id):
        
        global log_df
        # Get the name corresponding to the ID
        name = df.loc[df['ID'] == id, 'Name'].values[0]
        
        current_time = datetime.now()
        date_str = current_time.strftime("%Y-%m-%d")
        time_str = current_time.strftime("%H:%M:%S")
    
        # Create a new row to append
        new_entry = pd.DataFrame({'No.': [len(log_df) + 1,], 'ID': [id], 'Name': [name], 'Date': [date_str], 'Time': [time_str]})
        
        # Append the new row to the log DataFrame
        log_df = pd.concat([log_df, new_entry], ignore_index=True)
        
        # Write the updated DataFrame back to the Excel file
        log_df.to_excel(log_file_path, index=False)


while True: 
    
    imgBackground = cv2.imread(r"C:\Users\hoque\Desktop\OpenCV\VSCode\Files\background2.jpg")
    imgBackground = cv2.resize(imgBackground, (1000,500))

    success, img = cap.read()
    H,W,_ = img.shape
    roiy1, roiy2, roix1, roix2 = int(H*0.3), int(H-0.3*H) ,int(W*0.3), int(W-0.3*W) 
    x, y, w, h = roix1, roiy1, roix2-roix1, roiy2-roiy1 
    #cornerRect(img, [x,y,w,h])
    roi = img[roiy1:roiy2, roix1:roix2]

    #Graphical Addition 
    imgroi= cv2.resize(roi, (832-570,393-116))
    imgBackground[116:393, 570:832] = imgroi 

    results = decode(roi)

    if results: 
            for barcode in results: 

                encodedData = barcode.data.decode('utf-8')
                code = re.search(r'\d+', encodedData).group()
                code = int(code)
                if flag and counter==0: 
                    counter = 1
                    print(encodedData)

                    if code in id_list:
                         arduino.sendData([1])
                         color = (0,210,0)
                         log_event(code) 
                         textr =  "AUTHORIZED"
                         print("Success")
                    else: 
                         arduino.sendData([2])
                         color = (0,0,255)
                         textr =  "UNAUTHORIZED"
                         print("Failure")

                if counter!=0:
                    counter+=1
                    if counter>12:
                            counter=0

                pts = np.array([barcode.polygon], np.int32)
                pts = pts.reshape((-1,1,2))

                pts[:, 0, 0] += roix1  # Adjust x coordinates
                pts[:, 0, 1] += roiy1  # Adjust y coordinates

                # Calculate scale factors
                scale_x = (832 - 570) / (roix2 - roix1)
                scale_y = (393 - 116) / (roiy2 - roiy1)

                # Adjust the points
                pts2 = np.zeros_like(pts)
                pts2[:, 0, 0] = (pts[:, 0, 0] - roix1) * scale_x + 570
                pts2[:, 0, 1] = (pts[:, 0, 1] - roiy1) * scale_y + 116

               
                # putTextRect(img, text=textr, pos=(int(W*0.7), 30), scale=1.5, thickness=2, colorR=color)
                # putTextRect(imgBackground, text=textr, pos=(int(W*0.7), 30), scale=1.5, thickness=2, colorR=color)

                putTextRect(img, text=textr, pos=(int(roix1+55), roiy1-15), scale=1.5, thickness=2, colorR=color)
                putTextRect(imgBackground, text=textr, pos=(int(570+60), 116-20), scale=1.5, thickness=2, colorR=color)
                cv2.polylines(img, [pts], True, (0,0,255), thickness=3)
                cv2.polylines(imgBackground, [pts2], True, (0,0,255), thickness=3)
                flag = False
                
    else: 
        flag = True


    
    #fpsReader.update returns the current FPS and the updated image
    fps, img = fpsReader.update(img, pos=(20, 50),bgColor=(0, 0, 255), textColor=(255, 255, 255), scale=1.3, thickness=2)

    
    putTextRect(imgBackground, text="SCAN QR", pos=(int(570+75), 393+30), scale=1.5, thickness=2, colorR=(255,0,0))
    cornerRect(imgBackground, [570-5,116-5,262+5,277+5], rt=6, t=10,colorR=(0,0,0),colorC=(255,0,0),l=35)
    imgBackground= cv2.resize(imgBackground, (800,480))
    cv2.imshow("Grphics Added", imgBackground )
    
    cornerRect(img, [x,y,w,h], rt=2)
    putTextRect(img, text="SCAN QR", pos=(int(roix1+72), roiy2+25), scale=1.5, thickness=2, colorR=(255,0,0))
    cv2.imshow("QR Scanner", img)
   

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

    
